In [27]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
    #for filename in filenames:
        #print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

All the imports:

In [28]:
import pandas as pd
import matplotlib.pyplot as plt
import os
from PIL import Image
import numpy as np
from tqdm.auto import tqdm
import numpy as np
from sklearn.model_selection import GroupKFold
from sklearn.metrics import r2_score
from lightgbm import LGBMRegressor
from lightgbm import early_stopping, log_evaluation
from sklearn.model_selection import KFold
from sklearn.preprocessing import OneHotEncoder
import cv2
from sklearn.model_selection import train_test_split

print("Done")

Done


Understanding and exploring data

In [29]:
BASE_IMG_DIR = "/kaggle/input/csiro-biomass/train"
TEST_IMG_DIR = "/kaggle/input/csiro-biomass/test"

long = pd.read_csv("/kaggle/input/csiro-biomass/train.csv")
long.head(10)

,sample_id,image_path,Sampling_Date,State,Species,Pre_GSHH_NDVI,Height_Ave_cm,target_name,target
0,ID1011485656__Dry_Clover_g,train/ID1011485656.jpg,2015/9/4,Tas,Ryegrass_Clover,0.62,4.6667,Dry_Clover_g,0.0000
1,ID1011485656__Dry_Dead_g,train/ID1011485656.jpg,2015/9/4,Tas,Ryegrass_Clover,0.62,4.6667,Dry_Dead_g,31.9984
2,ID1011485656__Dry_Green_g,train/ID1011485656.jpg,2015/9/4,Tas,Ryegrass_Clover,0.62,4.6667,Dry_Green_g,16.2751
3,ID1011485656__Dry_Total_g,train/ID1011485656.jpg,2015/9/4,Tas,Ryegrass_Clover,0.62,4.6667,Dry_Total_g,48.2735
4,ID1011485656__GDM_g,train/ID1011485656.jpg,2015/9/4,Tas,Ryegrass_Clover,0.62,4.6667,GDM_g,16.2750
5,ID1012260530__Dry_Clover_g,train/ID1012260530.jpg,2015/4/1,NSW,Lucerne,0.55,16.0000,Dry_Clover_g,0.0000
6,ID1012260530__Dry_Dead_g,train/ID1012260530.jpg,2015/4/1,NSW,Lucerne,0.55,16.0000,Dry_Dead_g,0.0000
7,ID1012260530__Dry_Green_g,train/ID1012260530.jpg,2015/4/1,NSW,Lucerne,0.55,16.0000,Dry_Green_g,7.6000
8,ID1012260530__Dry_Total_g,train/ID1012260530.jpg,2015/4/1,NSW,Lucerne,0.55,16.0000,Dry_Total_g,7.6000
9,ID1012260530__GDM_g,train/ID1012260530.jpg,2015/4/1,NSW,Lucerne,0.55,16.0000,GDM_g,7.6000


data long format -> wide format. one row per img

In [30]:
long["image_id"] = long["sample_id"].str.split("__").str[0]

wide = long.pivot_table(
    index=['image_id', 'image_path', 'Sampling_Date', 'State', 'Species', 'Pre_GSHH_NDVI', 'Height_Ave_cm'],
    columns='target_name',
    values='target'
).reset_index()

wide.columns.name = None
wide = wide.rename_axis(None, axis=1)

targets = ["Dry_Green_g","Dry_Dead_g","Dry_Clover_g","GDM_g","Dry_Total_g"]

print("Rows (images):", wide.shape[0])
print("Columns:", wide.columns.tolist())
wide.head(10)


Rows (images): 357
Columns: ['image_id', 'image_path', 'Sampling_Date', 'State', 'Species', 'Pre_GSHH_NDVI', 'Height_Ave_cm', 'Dry_Clover_g', 'Dry_Dead_g', 'Dry_Green_g', 'Dry_Total_g', 'GDM_g']


,image_id,image_path,Sampling_Date,State,Species,Pre_GSHH_NDVI,Height_Ave_cm,Dry_Clover_g,Dry_Dead_g,Dry_Green_g,Dry_Total_g,GDM_g
0,ID1011485656,train/ID1011485656.jpg,2015/9/4,Tas,Ryegrass_Clover,0.62,4.6667,0.0000,31.9984,16.2751,48.2735,16.2750
1,ID1012260530,train/ID1012260530.jpg,2015/4/1,NSW,Lucerne,0.55,16.0000,0.0000,0.0000,7.6000,7.6000,7.6000
2,ID1025234388,train/ID1025234388.jpg,2015/9/1,WA,SubcloverDalkeith,0.38,1.0000,6.0500,0.0000,0.0000,6.0500,6.0500
3,ID1028611175,train/ID1028611175.jpg,2015/5/18,Tas,Ryegrass,0.66,5.0000,0.0000,30.9703,24.2376,55.2079,24.2376
4,ID1035947949,train/ID1035947949.jpg,2015/9/11,Tas,Ryegrass,0.54,3.5000,0.4343,23.2239,10.5261,34.1844,10.9605
5,ID1036339023,train/ID1036339023.jpg,2015/9/30,Vic,Phalaris_Clover,0.82,7.0000,23.0755,2.6135,32.1910,57.8800,55.2665
6,ID1049634115,train/ID1049634115.jpg,2015/7/2,Vic,Ryegrass_Clover,0.72,2.0000,1.5083,3.0167,13.5750,18.1000,15.0833
7,ID1051144034,train/ID1051144034.jpg,2015/9/1,WA,SubcloverLosa,0.80,1.0000,55.3200,0.0000,0.0000,55.3200,55.3200
8,ID1052620238,train/ID1052620238.jpg,2015/5/18,Tas,Ryegrass,0.68,5.0000,0.0000,11.2291,20.1707,31.3998,20.1707
9,ID105271783,train/ID105271783.jpg,2015/6/30,Vic,Phalaris_Clover,0.86,4.0000,5.2698,8.5635,27.6667,41.5000,32.9365


In [31]:
print("Total images:", len(wide))
print("Missing values per column:\n", wide.isna().sum())

display(wide[targets].describe().T)
print(wide.groupby('Species')['Dry_Total_g'].mean().sort_values(ascending=False).head(10))

Total images: 357
Missing values per column:
 image_id         0
image_path       0
Sampling_Date    0
State            0
Species          0
Pre_GSHH_NDVI    0
Height_Ave_cm    0
Dry_Clover_g     0
Dry_Dead_g       0
Dry_Green_g      0
Dry_Total_g      0
GDM_g            0
dtype: int64


,count,mean,std,min,25%,50%,75%,max
Dry_Green_g,357.0,26.624722,25.401232,0.00,8.8000,20.8000,35.0834,157.9836
Dry_Dead_g,357.0,12.044548,12.402007,0.00,3.2000,7.9809,17.6378,83.8407
Dry_Clover_g,357.0,6.649692,12.117761,0.00,0.0000,1.4235,7.2429,71.7865
GDM_g,357.0,33.274414,24.935822,1.04,16.0261,27.1082,43.6757,157.9836
Dry_Total_g,357.0,45.318097,27.984015,1.04,25.2715,40.3000,57.8800,185.7000


Species
Phalaris                                                       100.262500
Fescue                                                          86.317857
Phalaris_Clover_Ryegrass_Barleygrass_Bromegrass                 55.557143
Lucerne                                                         54.654545
Phalaris_Clover                                                 51.409286
Mixed                                                           47.563000
Fescue_CrumbWeed                                                46.550000
Phalaris_BarleyGrass_SilverGrass_SpearGrass_Clover_Capeweed     39.663636
Ryegrass_Clover                                                 38.889031
Ryegrass                                                        37.503361
Name: Dry_Total_g, dtype: float64


In [32]:
def get_img_info(full_img_path, downscale=(500,250)):
    im = Image.open(full_img_path).convert("RGB")
    im.thumbnail(downscale)
    arr = np.asarray(im)/255.0
    
    mean_rgb = arr.mean(axis=(0,1))
    std_rgb = arr.std(axis=(0,1))
    green_ratio = mean_rgb[1] / (mean_rgb.sum() + 1e-9)
    hsv = cv2.cvtColor((arr*255).astype(np.uint8), cv2.COLOR_RGB2HSV)
    brightness = hsv[:,:,2].mean()/255
    saturation = hsv[:,:,1].mean()/255
    gray = (0.2989*arr[:,:,0] + 0.587*arr[:,:,1] + 0.114*arr[:,:,2])
    lap_var = cv2.Laplacian((gray*255).astype(np.uint8), cv2.CV_64F).var()
    
    return {
        "meanR": mean_rgb[0], "meanG": mean_rgb[1], "meanB": mean_rgb[2],
        "stdR": std_rgb[0], "stdG": std_rgb[1], "stdB": std_rgb[2],
        "green_ratio": green_ratio,
        "brightness": brightness, "saturation": saturation,
        "lap_var": lap_var
    }

# Loop through all train images
rows = []
for i in tqdm(wide['image_path'].unique(), desc="Extracting image features"):
    fname= os.path.basename(i)
    full_path = os.path.join(BASE_IMG_DIR, fname)
    feats = get_img_info(full_path)
    feats['image_id'] = os.path.basename(i).split('.')[0]
    rows.append(feats)

img_feats_df = pd.DataFrame(rows)
img_feats_df.to_csv("image_features.csv", index=False)
print("Saved image_features.csv")


Extracting image features:   0%|          | 0/357 [00:00<?, ?it/s]

Saved image_features.csv


In [33]:
print("wide:", wide.shape)
print("img_feats:", img_feats_df.shape)

merged = wide.merge(img_feats_df, on="image_id", how="left")

print("merged:", merged.shape)
print("Missing feature rows:", merged["meanR"].isna().sum())

merged.to_csv("training_table.csv", index=False)
print("Saved training_table.csv")


wide: (357, 12)
img_feats: (357, 11)
merged: (357, 22)
Missing feature rows: 0
Saved training_table.csv


merged now contains data from csv and also from the images

In [34]:
df = pd.read_csv("training_table.csv")
print(df.columns)
print(df.shape)
print(df.head())

# targets = ["Dry_Green_g","Dry_Dead_g","Dry_Clover_g","GDM_g","Dry_Total_g"]
target_weights = {
    "Dry_Green_g": 0.1,
    "Dry_Dead_g":  0.1,
    "Dry_Clover_g":0.1,
    "GDM_g":       0.2,
    "Dry_Total_g": 0.5
}


cat_cols = ["State", "Species"]
encoder = OneHotEncoder(sparse_output=False, handle_unknown="ignore")
encoded_cats = encoder.fit_transform(df[cat_cols])
cat_feature_names = encoder.get_feature_names_out(cat_cols)
df_cat = pd.DataFrame(encoded_cats, columns=cat_feature_names)

numeric_cols = [
    c for c in df.columns
    if c not in ["image_id","image_path","Sampling_Date"] + cat_cols + targets
]
df_num = df[numeric_cols].reset_index(drop=True)
X = pd.concat([df_num, df_cat], axis=1)
#print("Final feature matrix shape:", X.shape)

def weighted_r2(y_true, y_pred, weights):
    w = np.array(weights, dtype=float)
    y_true = np.array(y_true, dtype=float)
    y_pred = np.array(y_pred, dtype=float)
    mu = np.sum(w * y_true) / (np.sum(w) + 1e-12)
    sse = np.sum(w * (y_true - y_pred)**2)
    sst = np.sum(w * (y_true - mu)**2) + 1e-12
    return 1 - sse / sst


# KFold
kf = KFold(n_splits=5, shuffle=True, random_state=42)
fold_scores= []
models = {t: None for t in targets}

for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
    print(f"Fold {fold}")

    val_preds_per_target = {}
    val_trues_per_target = {}
    val_weights_per_target = {}
    
    #y = df[target].values
    
    for target in targets:
        y = df[target].values
        y_train, y_val = y[train_idx], y[val_idx]

        model = LGBMRegressor(
            n_estimators=2000,
            learning_rate=0.03,
            max_depth=6,
            num_leaves=31,
            #min_data_in_leaf=10,
            #min_gain_to_split=0.001,
            subsample=0.8,
            colsample_bytree=0.8,
            random_state=42
        )

        model.fit(
            X.iloc[train_idx], y_train,
            eval_set=[(X.iloc[val_idx], y_val)],
            eval_metric='l2',
            callbacks=[early_stopping(stopping_rounds=50), log_evaluation(0)]
        )

        preds_val = model.predict(X.iloc[val_idx])
        val_preds_per_target[target] = preds_val
        val_trues_per_target[target] = y_val
        val_weights_per_target[target] = np.full_like(y_val, fill_value=target_weights[target], dtype=float)


    # After all targets predicted for this fold, concatenate in the same order for global metric
    y_true_all = np.concatenate([val_trues_per_target[t] for t in targets])
    y_pred_all = np.concatenate([val_preds_per_target[t] for t in targets])
    weights_all = np.concatenate([val_weights_per_target[t] for t in targets])

    global_r2 = weighted_r2(y_true_all, y_pred_all, weights_all)
    print(f"  Global weighted R² (fold {fold}): {global_r2:.6f}")
    fold_scores.append(global_r2)

print("CV global weighted R² per fold:", fold_scores)
print("Mean global weighted R²:", np.mean(fold_scores))


Index(['image_id', 'image_path', 'Sampling_Date', 'State', 'Species',
       'Pre_GSHH_NDVI', 'Height_Ave_cm', 'Dry_Clover_g', 'Dry_Dead_g',
       'Dry_Green_g', 'Dry_Total_g', 'GDM_g', 'meanR', 'meanG', 'meanB',
       'stdR', 'stdG', 'stdB', 'green_ratio', 'brightness', 'saturation',
       'lap_var'],
      dtype='object')
(357, 22)
       image_id              image_path Sampling_Date State  \
0  ID1011485656  train/ID1011485656.jpg      2015/9/4   Tas   
1  ID1012260530  train/ID1012260530.jpg      2015/4/1   NSW   
2  ID1025234388  train/ID1025234388.jpg      2015/9/1    WA   
3  ID1028611175  train/ID1028611175.jpg     2015/5/18   Tas   
4  ID1035947949  train/ID1035947949.jpg     2015/9/11   Tas   

             Species  Pre_GSHH_NDVI  Height_Ave_cm  Dry_Clover_g  Dry_Dead_g  \
0    Ryegrass_Clover           0.62         4.6667        0.0000     31.9984   
1            Lucerne           0.55        16.0000        0.0000      0.0000   
2  SubcloverDalkeith           0.38       

In [39]:
final_models = {}
for target in targets:
    y = df[target].values
    Xtrain, Xval, ytrain, yval = train_test_split(X, y, test_size=0.1, random_state=42)
    model = LGBMRegressor(
        n_estimators=5000,
        learning_rate=0.03,
        num_leaves=31,
        max_depth=6)
    model.fit(
        Xtrain,
        ytrain,
        eval_set=[(Xval, yval)],
        callbacks=[early_stopping(stopping_rounds=50),
        log_evaluation(0)]
    )
    final_models[target] = model

# load test
test_df = pd.read_csv("/kaggle/input/csiro-biomass/test.csv")
test_df['image_id'] = test_df['sample_id'].str.split('__').str[0]

for col in ["Sampling_Date","State","Species"]:
    test_df[col] = "missing"    # categorical defaults

for col in ["Pre_GSHH_NDVI","Height_Ave_cm"]:
    if col in df.columns:
        test_df[col] = df[col].median()
    else:
        test_df[col] = 0.0

rows = []
for p in tqdm(test_df['image_path'].unique(), desc="Extracting image features (test)"):
    fname = os.path.basename(p)
    full_path = os.path.join(TEST_IMG_DIR, fname)
    feats = get_img_info(full_path)
    feats['image_id'] = fname.split('.')[0]
    rows.append(feats)

test_img_df = pd.DataFrame(rows)
test_images = test_df[['image_id','image_path','Sampling_Date','State','Species','Pre_GSHH_NDVI','Height_Ave_cm']].drop_duplicates(subset=['image_id']).reset_index(drop=True)
test_merged = test_images.merge(test_img_df, on='image_id', how='left')

test_merged[cat_cols] = test_merged[cat_cols].fillna("missing").astype(str)
test_cat = encoder.transform(test_merged[cat_cols])
test_cat_df = pd.DataFrame(test_cat, columns=cat_feature_names)
test_num = test_merged[numeric_cols].reset_index(drop=True)
test_X = pd.concat([test_num, test_cat_df], axis=1)
test_X = test_X.reindex(columns=X.columns, fill_value=0)

# final submission
rows = []
for idx, row in test_X.reset_index(drop=True).iterrows():
    image_id = test_merged.loc[idx, "image_id"]   # --- FIX: use test_merged, not test_df
    for target in targets:
        pred = final_models[target].predict(row.values.reshape(1,-1))[0]
        sample_id = f"{image_id}__{target}"
        rows.append({"sample_id": sample_id, "target": float(pred)})

submission = pd.DataFrame(rows)
submission.to_csv("submission.csv", index=False)
print("Saved submission.csv")

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000104 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1193
[LightGBM] [Info] Number of data points in the train set: 321, number of used features: 21
[LightGBM] [Info] Start training from score 27.279204
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 50 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -in

Extracting image features (test):   0%|          | 0/1 [00:00<?, ?it/s]

Saved submission.csv
